In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from data.unlabeled import OpenWebTextConfig, TiktokenTokenizer
from model.gpt2 import GPT
from train import OWTData, RandomSubsetSampler, save_model_checkpoints
import numpy as np
from fastai.text.all import *
from torch.utils.data import DataLoader

/home/agrawalp2/miniconda3/envs/tinylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from model.customLearner import customLearner

In [4]:
tokenizer = TiktokenTokenizer()

In [5]:

bs = 16
block_size = 512
valid_sampler_size = 1000 #how many samples to use for validation. This is only used to check if validation loss is better than best_valid_loss, so that a checkpoint can be saved. Karpathy uses 200 random points



model = GPT(block_size=block_size)

train_ds = OWTData(OpenWebTextConfig().default_cache_dir/'train.bin'    ,block_size=block_size, dtype = tokenizer._get_numpy_dtype())
valid_ds = OWTData(OpenWebTextConfig().default_cache_dir/'val.bin'      ,block_size=block_size, dtype = tokenizer._get_numpy_dtype())
                    
train_dl = DataLoader(train_ds, batch_size=bs)

valid_dl = DataLoader(valid_ds, batch_size=2*bs,
                        sampler = RandomSubsetSampler(valid_ds, subset_size=valid_sampler_size),
                        )


dls = DataLoaders(train_dl, valid_dl)
dls.c = model.head.out_features


check_and_save_model = save_model_checkpoints(dir = 'checkpoints', 
                                                model_name = str(model), 
                                                checkpoint_name = 'gpt2', 
                                                every_iters = 10000)

learn = customLearner(dls, 
                model, 
                loss_func=CrossEntropyLossFlat(), 
                metrics=[accuracy, Perplexity()],
                cbs = [check_and_save_model],
                ).to_bf16()

In [12]:
learn.dls.device 

AttributeError: 'DataLoader' object has no attribute 'device'

In [6]:
learn.fit_one_cycle(1, 1e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: 

In [16]:
learn.load('gpt2', device = 'cuda:0')

In [17]:
learn.fit_one_cycle(1, 1e-4)

epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: 